In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Verkhnevilyuysk,63.45,120.32,-3.86,89,14,4.56,RU,2020-11-09 01:25:57
1,1,Asau,46.43,26.40,42.01,100,0,4.70,RO,2020-11-09 01:25:57
2,2,Ushuaia,-54.80,-68.30,44.60,52,75,27.51,AR,2020-11-09 01:21:21
3,3,Mataura,-46.19,168.86,57.00,69,93,8.01,NZ,2020-11-09 01:21:31
4,4,Punta Arenas,-53.15,-70.92,42.80,75,0,17.22,CL,2020-11-09 01:21:06


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [13]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig


Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [20]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,Port Blair,11.67,92.75,82.18,79,14,10.78,IN,2020-11-09 01:21:27
18,18,Atuona,-9.80,-139.03,77.74,65,100,12.57,PF,2020-11-09 01:26:00
23,23,Marang,5.21,103.21,77.00,94,40,2.24,MY,2020-11-09 01:26:00
26,26,Ruteng,-8.61,120.47,76.41,63,95,2.62,ID,2020-11-09 01:26:01
28,28,Doha,25.29,51.53,81.00,83,0,2.24,QA,2020-11-09 01:24:40
32,32,Cockburn Town,21.46,-71.14,81.59,81,100,26.64,TC,2020-11-09 01:26:01
37,37,Sao Filipe,14.90,-24.50,79.83,73,0,8.34,CV,2020-11-09 01:26:02
40,40,Grand-Lahou,5.14,-5.02,78.80,85,80,9.08,CI,2020-11-09 01:26:02
44,44,Luba,3.46,8.55,78.24,79,72,4.63,GQ,2020-11-09 01:26:03
46,46,Thinadhoo,0.53,72.93,83.71,75,84,11.54,MV,2020-11-09 01:21:09


In [22]:
preferred_cities_df.count()

City_ID       168
City          168
Lat           168
Lng           168
Max Temp      168
Humidity      168
Cloudiness    168
Wind Speed    168
Country       168
Date          168
dtype: int64

In [23]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
17,Port Blair,IN,82.18,11.67,92.75,
18,Atuona,PF,77.74,-9.80,-139.03,
23,Marang,MY,77.00,5.21,103.21,
26,Ruteng,ID,76.41,-8.61,120.47,
28,Doha,QA,81.00,25.29,51.53,
32,Cockburn Town,TC,81.59,21.46,-71.14,
37,Sao Filipe,CV,79.83,14.90,-24.50,
40,Grand-Lahou,CI,78.80,5.14,-5.02,
44,Luba,GQ,78.24,3.46,8.55,
46,Thinadhoo,MV,83.71,0.53,72.93,


In [24]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [29]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
17,Port Blair,IN,82.18,11.67,92.75,Fortune Resort Bay Island
18,Atuona,PF,77.74,-9.80,-139.03,Villa Enata
23,Marang,MY,77.00,5.21,103.21,DBarakah Homestay
26,Ruteng,ID,76.41,-8.61,120.47,Hotel FX72 Ruteng
28,Doha,QA,81.00,25.29,51.53,Doha Marriott Hotel
32,Cockburn Town,TC,81.59,21.46,-71.14,Osprey Beach Hotel
37,Sao Filipe,CV,79.83,14.90,-24.50,Tortuga B&B
40,Grand-Lahou,CI,78.80,5.14,-5.02,Hotel Beatitude de Grand Lahou
44,Luba,GQ,78.24,3.46,8.55,
46,Thinadhoo,MV,83.71,0.53,72.93,


In [31]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [34]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [35]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))